In [7]:
import numpy
from combind_utils.bounding_box_utils import read_bounding_box, is_within_bounding_box
from combind_utils.atoms_utils import get_atoms_coordinates, get_ligand_protein_path, read_poses_and_calculate_rmsd
from schrodinger.structutils import rmsd

In [8]:
import glob
protein_path = glob.glob("/home/pc/Documents/combind_fragment/combind_fragment/fragment_dataset_redocking/*")
import os

from collections import defaultdict
ligands_dict = defaultdict(list)
for each_protein_path in protein_path:
    reference_protein_path = os.path.join(each_protein_path, "structures", "grids")
    #list folders under the path
    reference_protein_path = os.listdir(reference_protein_path)
    if len(reference_protein_path) > 1:
        assert False, "More than one reference protein path found"
    
    reference_protein_name = reference_protein_path[0].split("/")[-1]
    ligands_path = glob.glob(f"{each_protein_path}/structures/ligands/*.mae")
    ligands_dict[(reference_protein_name,os.path.join(each_protein_path))] = ligands_path






In [9]:
import schrodinger
#try to read the mae file
from schrodinger.structure import StructureReader
from tqdm import tqdm
from schrodinger.structutils import rmsd

protein_to_ligands = {}

for reference_ligand_name, ligand_base_path in tqdm(ligands_dict):
    
    # Path to bounding box file: <ligand_base_path>/structures/grids/<reference_ligand_name>/<reference_ligand_name>.in
    boxing_box_path = os.path.join(ligand_base_path, "structures", "grids", reference_ligand_name,f"{reference_ligand_name}.in")
    bounding_box = read_bounding_box(boxing_box_path)
    
    # Path to reference protein structure: <ligand_base_path>/structures/proteins/<reference_ligand_name>_prot.mae
    reference_protein_path = os.path.join(ligand_base_path, "structures", "proteins", f"{reference_ligand_name}_prot.mae")
    pocket_atoms = []
    
    # Read protein structure from MAE file and extract atoms within the binding pocket
    # This code reads a protein structure from a MAE file and identifies atoms that lie within 
    # a predefined bounding box region representing the binding pocket. For each atom in the protein,
    # it checks if the atom's coordinates fall within the box boundaries. If an atom is within the 
    # bounding box, it is considered part of the binding pocket and added to the pocket_atoms list.
    with StructureReader(reference_protein_path) as reader:
        for model in reader:
            # Iterate through atoms in the protein model
            for atom in model.atom:
                # Get x,y,z coordinates of atom
                x, y, z = get_atoms_coordinates(atom)
                # Check if atom is within the defined bounding box region
                if is_within_bounding_box([x, y, z], bounding_box, box_type = "outer_box"):
                    pocket_atoms.append(atom)
        
                
    


100%|██████████| 44/44 [00:00<00:00, 105.60it/s]


In [10]:


# Path to the predicted poses from Glide docking
poses_pred_path = "/home/pc/Documents/combind_fragment/combind_fragment/fragment_dataset_redocking/A5H660/docking/*/*_pv.maegz"
#
#poses_pred_path = "/home/pc/Documents/combind_fragment/combind_fragment_test/A5H660/docking_re_smiles/*/*_pv.maegz"
poses_pred_path = glob.glob(poses_pred_path)

rmsd_values = {}
poses_pred_structures_glide = {}
corrected_poses = 0

for pose_pred_path in tqdm(poses_pred_path):
    reference_ligand_name = pose_pred_path.split("/")[-1].split("_pv")[0].split("-to-")[1].lower()
    ligand_name = pose_pred_path.split("/")[-1].split("_pv")[0].split("-to-")[0].lower()
    
    pose_true_path = os.path.join("/",*pose_pred_path.split("/")[:-3], "structures", "ligands", f"{ligand_name}.mae")
    #find rmsd between pose_pred and pose_true
    poses_pred_structure_glide, rmsd_value = read_poses_and_calculate_rmsd(pose_pred_path, pose_true_path, reference_ligand_name)
    rmsd_values[ligand_name] = rmsd_value
    poses_pred_structures_glide[ligand_name] = poses_pred_structure_glide
    corrected_poses += (rmsd_value < 2).sum()
    

100%|██████████| 5/5 [00:00<00:00, 10.50it/s]


In [12]:
pose_true_path = "/home/pc/Documents/combind_fragment/combind_fragment/fragment_dataset/A5H660/structures/ligands/6htg_lig.mae"
smallest_index = rmsd_values["6htg_lig"].argmin()
strcuture_fragment = list(poses_pred_structures_glide["6htg_lig"].keys())[smallest_index]
#save the structure to pdb file
strcuture_fragment.write("structure_fragment.pdb")

poses_true = next(StructureReader(pose_true_path))
#save to pdb file
poses_true.write("pose_true.pdb")
#convert to smiles
from rdkit import Chem
from rdkit.Chem import AllChem
mol = Chem.MolFromPDBFile("pose_true.pdb")
mol = Chem.RemoveAllHs(mol)  # Remove hydrogens for cleaner SMILES
AllChem.SanitizeMol(mol)     # Sanitize the molecule

# Convert to SMILES
smiles = Chem.MolToSmiles(mol)


#smiles = Chem.MolToSmiles(poses_true)


In [68]:
import pandas as pd
df = pd.read_csv("/home/pc/Documents/combind_fragment/combind_fragment/fragment_dataset/A5H660/docking/6HTG_lig-to-4bz6/6HTG_lig-to-4bz6_ifp_rd1_raw.csv")

df_best_predict_pose = df[df["pose"] == smallest_index]
#find number of hydrogen bond
df_best_predict_pose["hydrogen"].value_counts()

hydrogen
H2     2
H1     1
H5     1
HE2    1
HH     1
Name: count, dtype: int64

In [64]:
df.head()

,pose,label,protein_res,protein_atom,ligand_atom,dist,angle,hydrogen,vdw
0,0,contact,A:100:ASP:,CA,Cl1,5.784160,NaN,NaN,3.45
1,0,contact,A:100:ASP:,CB,C4,5.377163,NaN,NaN,3.40
2,0,contact,A:100:ASP:,CB,C5,5.361453,NaN,NaN,3.40
3,0,contact,A:100:ASP:,CB,Cl1,4.469744,NaN,NaN,3.45
4,0,contact,A:100:ASP:,CG,C4,5.370163,NaN,NaN,3.40


In [56]:
rmsd_values.keys()

dict_keys(['4bz6_lig', '6htg_lig', '6hsf_lig', '6hsz_lig', '6ht8_lig'])

In [11]:
#plot rmsd values
from matplotlib import pyplot as plt
plt.hist(rmsd_values["6htg_lig"], bins=100)
plt.show()


In [67]:
model_atom_list = list(model.atom)
pose_true_atom_list = list(poses_true.atom)
#save model and pose_true to pdb file
model.write("model.pdb")
poses_true.write("pose_true.pdb")